In [1]:
import pandas as pd
import numpy as np
from dataclasses import dataclass

In [2]:
DATABASE_PATH = './data/spy_2020_2022.sqlite'
TABLE_NAME = 'spy_2020_2022'

In [ ]:
import sqlite3
from IPython.display import display
from pathlib import Path

"""
SQLite connection + quick sanity check
"""
db_path = Path(DATABASE_PATH)
if not db_path.exists():
    raise FileNotFoundError(f"SQLite file not found: {db_path.resolve()}")

conn = sqlite3.connect(str(db_path))
try:
    tables = pd.read_sql_query(
        "SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;",
        conn,
    )
    display(tables)

    if TABLE_NAME not in set(tables["name"].tolist()):
        raise ValueError(
            f"Table '{TABLE_NAME}' not found. Available tables: {tables['name'].tolist()}"
        )

    print(f"Reading first 5 rows from table: {TABLE_NAME}")
    df_head = pd.read_sql_query(f"SELECT * FROM [{TABLE_NAME}] LIMIT 10;", conn)

    pd.set_option("display.width", None)
    display(df_head)
finally:
    conn.close()

,[QUOTE_UNIXTIME],[QUOTE_READTIME],[QUOTE_DATE],[QUOTE_TIME_HOURS],[UNDERLYING_LAST],[EXPIRE_DATE],[EXPIRE_UNIX],[DTE],[C_DELTA],[C_GAMMA],...,[P_LAST],[P_DELTA],[P_GAMMA],[P_VEGA],[P_THETA],[P_RHO],[P_IV],[P_VOLUME],[STRIKE_DISTANCE],[STRIKE_DISTANCE_PCT]
0,1630526400,2021-09-01 16:00,2021-09-01,16.0,451.85,2021-09-01,1630526400,0.0,1.0,0.0,...,0.01,-0.00024,0.00000,0.00004,-0.00509,0.0,3.41249,3.0,181.9,0.402
1,1630526400,2021-09-01 16:00,2021-09-01,16.0,451.85,2021-09-01,1630526400,0.0,1.0,0.0,...,0.01,-0.00063,0.00003,0.00019,-0.00480,0.0,3.29737,0.0,176.9,0.391
2,1630526400,2021-09-01 16:00,2021-09-01,16.0,451.85,2021-09-01,1630526400,0.0,1.0,0.0,...,0.01,0.00000,0.00000,-0.00023,-0.00534,0.0,3.18330,11.0,171.9,0.380
3,1630526400,2021-09-01 16:00,2021-09-01,16.0,451.85,2021-09-01,1630526400,0.0,1.0,0.0,...,0.01,-0.00016,0.00004,0.00038,-0.00532,0.0,3.07217,50.0,166.9,0.369
4,1630526400,2021-09-01 16:00,2021-09-01,16.0,451.85,2021-09-01,1630526400,0.0,1.0,0.0,...,0.01,-0.00068,0.00000,0.00045,-0.00515,0.0,2.96230,0.0,161.9,0.358
5,1630526400,2021-09-01 16:00,2021-09-01,16.0,451.85,2021-09-01,1630526400,0.0,1.0,0.0,...,0.04,-0.00052,-0.00003,0.00040,-0.00495,0.0,2.85433,1.0,156.9,0.347
6,1630526400,2021-09-01 16:00,2021-09-01,16.0,451.85,2021-09-01,1630526400,0.0,1.0,0.0,...,0.01,-0.00030,-0.00001,0.00024,-0.00475,0.0,2.74842,3.0,151.9,0.336
7,1630526400,2021-09-01 16:00,2021-09-01,16.0,451.85,2021-09-01,1630526400,0.0,1.0,0.0,...,0.02,-0.00055,0.00006,0.00055,-0.00495,0.0,2.64392,0.0,146.9,0.325
8,1630526400,2021-09-01 16:00,2021-09-01,16.0,451.85,2021-09-01,1630526400,0.0,1.0,0.0,...,0.08,-0.00078,-0.00002,0.00004,-0.00536,0.0,2.54076,1.0,141.9,0.314
9,1630526400,2021-09-01 16:00,2021-09-01,16.0,451.85,2021-09-01,1630526400,0.0,1.0,0.0,...,0.00,-0.00048,-0.00001,0.00009,-0.00486,0.0,2.43886,NaN,136.9,0.303


**Ito's Lemma**

$$
d\big(f(t,X_t)\big)
=
\Bigg[
\frac{\partial f}{\partial t}(t,X_t)
+ a(t,X_t)\frac{\partial f}{\partial x}(t,X_t)
+ \frac12 b^2(t,X_t)\frac{\partial^2 f}{\partial x^2}(t,X_t)
\Bigg]dt
+ b(t,X_t)\frac{\partial f}{\partial x}(t,X_t)\,dW_t.
$$

$$
\text{Option value} = e^{-r(T-t)}\,\mathbb{E}\big[\text{payoff}(S_T)\big]
$$

For each discrete step:

$$
S_{t+\Delta t} = S_t \cdot \exp {\left( (r-\frac{1}{2}\sigma^2)\Delta t + \sigma \sqrt{\Delta t} Z \right)}
$$

Black-Scholes PDE
$$
dC =
$$

In [ ]:
def Payoff_European_Call(S_T, K):
    return max(S_T - K, 0)

def Payoff_European_Put(S_T, K):
    return max(K - S_T, 0)

class MonteCarlo_EuropeanOption:
    def __init__(self, S0, K, T, r, σ):
        """
        Calculate the `fair-price` of a European option using the Monte Carlo method
        Args:
            S0: initial stock price
            K: strike price
            T: time to maturity
            r: risk-free interest rate
            σ: volatility of the stock
        """
        self.S0 = S0
        self.K = K
        self.T = T
        self.r = r
        self.σ = σ

    def nextStep(S):
        pass

# This is a demo usage

UNDERLYING_LAST = 451.85 # S₀
STRIKE = 270.0 # K
DTE = 10 # Date to expiration (days)
P_IV = 0 # Volatility
RISK_FREE_RATE = 0.01 # r


**Ito's Lemma**
$$
d\big(f(t,X_t)\big)
=
\Bigg[
\frac{\partial f}{\partial t}(t,X_t)
+ a(t,X_t)\frac{\partial f}{\partial x}(t,X_t)
+ \frac12 b^2(t,X_t)\frac{\partial^2 f}{\partial x^2}(t,X_t)
\Bigg]dt
+ b(t,X_t)\frac{\partial f}{\partial x}(t,X_t)\,dW_t.
$$
**Black Scholes PDE**


$$
d\Pi =
\left(
V_t + \frac12 \sigma^2 S^2 V_{SS}
\right) dt
$$

$$
d\Pi = r\Pi dt = r(V - S V_S) dt
$$

